In [10]:
import duckdb
import pandas as pd

In [17]:
sql_query = '''
SELECT * from ref_model
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,count_star()
0,97


In [19]:
sql_query = '''
SELECT * FROM bronze_parking_violations LIMIT 3
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,vehicle_color,vehicle_year
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50.0,50,50,365910,T201,D,0825A,BX,NaN,BLACK,0
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,NaN,0,0,0,None,None,1036A,BK,True,GY,2003
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,NaN,0,0,0,None,None,0126P,QN,True,WHI,2022


In [22]:
sql_query = '''
SELECT * FROM silver_parking_violation_codes 
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,True,515
1,1,FAILURE TO DISPLAY BUS PERMIT,False,515
2,2,NO OPERATOR NAM/ADD/PH DISPLAY,True,515
3,2,NO OPERATOR NAM/ADD/PH DISPLAY,False,515
4,3,UNAUTHORIZED PASSENGER PICK-UP,True,515
...,...,...,...,...
189,97,VACANT LOT,False,45
190,98,OBSTRUCTING DRIVEWAY,True,95
191,98,OBSTRUCTING DRIVEWAY,False,95
192,99,OTHER,True,105


In [25]:
sql_query = '''
SELECT
    violation_code,
    SUM(fee_usd) AS total_revenue_usd
FROM
    silver_parking_violation_codes
GROUP BY
    violation_code
HAVING
    NOT(total_revenue_usd >= 1)
 
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,violation_code,total_revenue_usd
0,41,0.0


In [15]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violations_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
)
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
)
'''

with duckdb.connect('data/nyc_parking_violations.db') as con:
  con.sql(sql_query_import_1)
  con.sql(sql_query_import_2)




In [ ]:
sql_query_import_1 = '''
CREATE OR REPLACE TABLE parking_violations_codes AS
SELECT *
FROM read_csv_auto(
  'data/dof_parking_violation_codes.csv',
  normalize_names=True
)
'''

sql_query_import_2 = '''
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
  'data/parking_violations_issued_fiscal_year_2023_sample.csv',
  normalize_names=True
)
'''


In [27]:
sql_query = '''
show tables
'''

with duckdb.connect('data/prod_nyc_parking_violations.db') as con:
    display(con.sql(sql_query).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,gold_ticket_metrics
3,gold_vehicles_metrics
4,parking_violation_codes
5,parking_violations_2023
6,silver_violation_tickets
7,silver_violation_vehicles
